In [146]:
import pandas as pd
import numpy as np
import math

How to make this model better:
1) delete useless words - for some reason really hurt the model, helped fake detection, destroyed real detection
2) accutrate priors (how likely is an article fake/real before we check it?) - 0.5 is optimal it would seem, others are really one sided
3) better use of k? - doesnt change anything
4) use of titles?
5) different categories of news
6) have an interface that says "this is likely fake/real whatever" + if a word is highly fake the program is like omg!!! this is likely fake because fo this 

The below functions are used to clear NaN's from datasets and remove any non-alphanumerics from lists of strings resepctively.

In [147]:
def NaN_Destroyer(listToClear):
    clearedList = []
    for i in range(len(listToClear)):
        if(not pd.isnull(listToClear[i])):
            clearedList.append(listToClear[i])
    return clearedList

def Clear_Data(listToClear):
    LOWERCASE_FROM_CAPITAL = ord('a') - ord('A')
    for i in range(len(listToClear)):
        currStringAsList = []
        for j in range(len(listToClear[i])):
            currCharacter = listToClear[i][j]
            if(ord(currCharacter)>=ord('A') and ord(currCharacter)<=ord('Z')):
                newCharacter = chr(ord(currCharacter) + LOWERCASE_FROM_CAPITAL)
                currStringAsList.append(newCharacter)
            if((ord(currCharacter)>=ord('0') and ord(currCharacter)<=ord('9')) or (ord(currCharacter)>=ord('a') and ord(currCharacter)<=ord('z')) or ord(currCharacter)==ord(' ')):
                currStringAsList.append(currCharacter)
        newString = ""
        for j in range(len(currStringAsList)):
            newString+=currStringAsList[j]
            listToClear[i] = newString
    return listToClear





In [148]:
# Code useful for when updating articles.zip

# fake_df = pd.read_csv("FakeFromFakeAndTrue.csv")
# fakeTextPlaceholder = Clear_Data(list(fake_df['text']))
# fakeTitlesPlaceholder = Clear_Data(list(fake_df['title']))

# real_df = pd.read_csv("TrueFromFakeAndTrue.csv")
# realText = Clear_Data(list(real_df['text']))
# realTitles = Clear_Data(list(real_df["title"]))

# fakeText = []
# fakeTitles = []

# for i in range(len(realText)):
#     fakeText.append(fakeTextPlaceholder[i])
#     fakeTitles.append(fakeTitlesPlaceholder[i])

# dictArticles = {
#     "realTitle": realTitles,
#     "Real": realText,
#     "fakeTitle": fakeTitles,
#     "Fake": fakeText
# }

# df = pd.DataFrame(dictArticles)
# df = df.loc[:len(realText)-1]

# Used to create the articles.zip with the latest version of articles.
# compression_opts = dict(method='zip',
#                         archive_name='v3.csv')  
# df.to_csv('articles.zip', index=False,
#           compression=compression_opts)  




df = pd.read_csv("v3.csv")

# I cut the train/test point at 75%.
cutOffPoint = int(len(df)*0.75)

trainDF = df.loc[:cutOffPoint]
testDF = df.loc[cutOffPoint+1:]

fakeArticles = list(trainDF['Fake'])
realArticles = list(trainDF['Real'])

dictOfAllWords = {}

for article in fakeArticles:
    articleAsList = article.split()
    for word in articleAsList:
        if(word in dictOfAllWords):
            dictOfAllWords[word][1] += 1
        else:
            dictOfAllWords[word] = [0,1] 

for article in realArticles:
    articleAsList = article.split()
    for word in articleAsList:
        if(word in dictOfAllWords):
            dictOfAllWords[word][0] += 1
        else:
            dictOfAllWords[word] = [1,0] 

words = list(dictOfAllWords.keys())
values = list(dictOfAllWords.values())
countCurrWordinReal = []
countCurrWordinFake = []

for pair in values:
    countCurrWordinReal.append(pair[0])
    countCurrWordinFake.append(pair[1])

dictFrequency = {
    "Word": words,
    "Real": countCurrWordinReal,
    "Fake": countCurrWordinFake
}
word_freq = pd.DataFrame(dictFrequency)
word_freq



,Word,Real,Fake
0,donald,9664,14275
1,trump,40111,61274
2,just,3469,15217
3,couldn,16,1151
4,t,1031,30873
...,...,...,...
198472,2184,1,0
198473,11139,1,0
198474,chaar,1,0
198475,faite,1,0


In [149]:
countRealArticles = 0
countFakeArticles = 0
k = 1

for i in range(len(words)):
    countRealArticles += countCurrWordinReal[i]
    countFakeArticles += countCurrWordinFake[i]

probWordIsFoundInReal = []
probWordIsFoundInFake = []

for i in range(len(words)):
    probIsInReal = (countCurrWordinReal[i]+k)/(countRealArticles + 2*k)
    probWordIsFoundInReal.append(probIsInReal)
    probisInFake = (countCurrWordinFake[i]+k)/(countFakeArticles + 2*k)
    probWordIsFoundInFake.append(probisInFake)

dictProb = {
    "Word": words,
    "Real": probWordIsFoundInReal,
    "Fake": probWordIsFoundInFake
}
word_prob = pd.DataFrame(dictProb)
word_prob

,Word,Real,Fake
0,donald,1.543931e-03,2.217535e-03
1,trump,6.407672e-03,9.518036e-03
2,just,5.543135e-04,2.363859e-03
3,couldn,2.715657e-06,1.789437e-04
4,t,1.648563e-04,4.795754e-03
...,...,...,...
198472,2184,3.194890e-07,1.553331e-07
198473,11139,3.194890e-07,1.553331e-07
198474,chaar,3.194890e-07,1.553331e-07
198475,faite,3.194890e-07,1.553331e-07


In [150]:
wordRealness = []
wordFakeness = []

articleIsReal = 0.5

for i in range(len(words)):
    currentRealness = (probWordIsFoundInReal[i]*articleIsReal)/((probWordIsFoundInReal[i]*articleIsReal) + probWordIsFoundInFake[i]*(1-articleIsReal))
    wordRealness.append(currentRealness)
    wordFakeness.append(1-currentRealness)

values = []
dictRealness = {}
for i in range(len(words)):
    values.append([wordRealness[i],wordFakeness[i]])
for i in range(len(words)):
    dictRealness[words[i]] = values[i]


In [154]:
testRealArticles = []
testFakeArticles = []
numberOfArticles = 2000

placeholderRealArticles = list(testDF['Real'])
placeholderFakeArticles = list(testDF['Fake'])
for i in range(numberOfArticles):
    testRealArticles.append(placeholderRealArticles[i])
    testFakeArticles.append(placeholderFakeArticles[i])

thinkRealIsRealCounter = 0
thinkFakeIsReal = 0

thinkFakeIsFakeCounter = 0
thinkRealIsFake = 0

for i in range(numberOfArticles):
    currRealArticle = testRealArticles[i].split()
    overallRealness = math.log(articleIsReal)
    overallFakeness = math.log(1-articleIsReal)
    for word in currRealArticle:
        if word in dictRealness:
            overallRealness += math.log(dictRealness[word][0])
            overallFakeness += math.log(dictRealness[word][1])
        else:
            continue
    if(overallRealness>overallFakeness):
        thinkRealIsRealCounter+=1
    else:
        thinkFakeIsReal+=1


for i in range(numberOfArticles):
    currFakeArticle = testFakeArticles[i].split()
    overallRealness = math.log(articleIsReal)
    overallFakeness = math.log(1-articleIsReal)
    for word in currFakeArticle:
        if word in dictRealness:
            overallRealness += math.log(dictRealness[word][0])
            overallFakeness += math.log(dictRealness[word][1])
        else:
            continue
    if(overallRealness<overallFakeness):
        thinkFakeIsFakeCounter+=1
    else:
        thinkRealIsFake+=1
    
print("Overall accuracy of the current model: " +
str(round((thinkRealIsRealCounter + thinkFakeIsFakeCounter)/
(thinkRealIsRealCounter + thinkFakeIsReal + thinkFakeIsFakeCounter + thinkRealIsFake)*100,2)) +
"%.")
print("Accuracy to detect a real article if it is real is " + str(round(thinkRealIsRealCounter/(thinkRealIsRealCounter+thinkFakeIsReal)*100,2))+"%.")
print("Accuracy to detect a fake article if it is fake is " + str(round(thinkFakeIsFakeCounter/(thinkFakeIsFakeCounter+thinkRealIsFake)*100,2))+"%.")

print("")

print("Think a real article is real: " + str(thinkRealIsRealCounter) + ".")
print("Think a fake article is real: " + str(thinkFakeIsReal) + ".")
print("Think a fake article is fake: " + str(thinkFakeIsFakeCounter) + ".")
print("Think a real article is fake: " + str(thinkRealIsFake) + ".")



Overall accuracy of the current model: 89.18%.
Accuracy to detect a real article if it is real is 98.8%.
Accuracy to detect a fake article if it is fake is 79.55%.

Think a real article is real: 1976.
Think a fake article is real: 24.
Think a fake article is fake: 1591.
Think a real article is fake: 409.


Here you can test a certain article's realness.

In [155]:
realArticle = "Protests that broke out over the weekend against Cuba’s government continued on Monday in some cities, including Tampa, Florida. Cuban anti-government activists said that over 100 people were arrested or missing as of Tuesday. Among those detained was Spanish journalist Camila Acosta, whose release was demanded by the Spanish government on Tuesday. The events also strained relations between the governments of the U.S. and Cuba, with President Joe Biden expressing support for the protesters, Cuban President Miguel Díaz-Canel accusing the U.S. of stoking social unrest in Cuba, and Secretary of State Anthony Blinken replying, “It would be a grievous mistake for the Cuban regime to interpret what is happening…as a result or product of anything the United States has done.”Voices in right-rated outlets used the protests to criticize progressive Democrats and tended to see the protests as a rejection of ideologies like communism and socialism. Voices in left- and center-rated outlets tended to focus more on material issues in Cuba like electricity cuts, food shortages and restrictions on free speech."
realArticle = Clear_Data(realArticle.split())

for word in realArticle:
    if word in stopWords:
        realArticle.remove(word)

overallRealness = math.log(articleIsReal)
overallFakeness = math.log(1-articleIsReal)
for word in realArticle:
        if word in dictRealness:
            overallRealness += math.log(dictRealness[word][0])
            overallFakeness += math.log(dictRealness[word][1])
        else:
            continue
if(overallRealness>overallFakeness):
    print("The article is real.")
else:
    print("The article is fake.")

realnessInThisArticle = []

for word in realArticle:
    if word in dictRealness:
        realnessInThisArticle.append(dictRealness[word][0])
wordAndProb = []
i=0
probabilities = []
for prob in realnessInThisArticle:
    probabilities.append(prob)

for word in realArticle:
    if word in dictRealness:
        realnessInThisArticle[i] = [word,realnessInThisArticle[i]]
        i+=1


probabilities.sort(reverse=True)
for currChance in probabilities:
    for pair in realnessInThisArticle:
        if currChance == pair[1]:
            break

print(overallRealness)
print(overallFakeness)


The article is real.
-70.4779351311706
-110.4285988393189
